# Лабораторная № 1. Базовая обработка и анализ текста.

In [ ]:
import pandas as pd

#### Данные отзывов о банковских услугах

In [ ]:
data = pd.read_csv('/content/financial_review.csv')
data.head()

,bank_href,login,review,bank_name,site,date,message_href,score,status,user_href
0,/kredit-evropa-bank.html,NaN,"Брал кредит в данной организации, после погаше...",кредит европа банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN
1,/kredit-evropa-bank.html,NaN,"В 2011 году,мною была оформлена мега карта в Т...",кредит европа банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN
2,/kredit-evropa-bank.html,NaN,С родителями увидели рекламу вклада Осеннее пр...,кредит европа банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN
3,/kredit-evropa-bank.html,NaN,Связать три года своей жизни с этим непорядочн...,кредит европа банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN
4,/kredit-evropa-bank.html,NaN,"Пользуюсь месяц карточкой Card Plus, сегодня п...",кредит европа банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN


#### Выберите один из банков (поле “bank_name”), отфильтуйте отзывы только для рассматриваемого банка.

In [ ]:
data['bank_name'].value_counts()

втб                             2603
сбербанк                        2481
тинькофф банк                   1476
почта банк                      1264
альфа-банк                      1260
                                ... 
национальный банк сбережений       1
электронный платежный сервис       1
внешфинбанк                        1
металлург                          1
банк развитие-столица              1
Name: bank_name, Length: 475, dtype: int64

In [ ]:
data_bank = data.loc[data['bank_name'] == 'тинькофф банк']
data_bank.head()

,bank_href,login,review,bank_name,site,date,message_href,score,status,user_href
2487,/tinkoff-kreditnye-sistemy.html,NaN,"С финансами было все не плохо,была работа у му...",тинькофф банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN
2488,/tinkoff-kreditnye-sistemy.html,NaN,Вчера закрывал задолженность по карте Тинькофф...,тинькофф банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN
2489,/tinkoff-kreditnye-sistemy.html,NaN,"Вот так! Я клиент банка уже почти год, имею и ...",тинькофф банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN
2490,/tinkoff-kreditnye-sistemy.html,NaN,"Хороший надежный Банк, клиентом которого я явл...",тинькофф банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN
2491,/tinkoff-kreditnye-sistemy.html,NaN,19.11.2016 у меня были украдены деньги с карты...,тинькофф банк,http://bankireview.ru,NaN,NaN,NaN,NaN,NaN


#### Выполните всю необходимую предварительную обработку текста (удаление лишних символов, стемминг/лемматизация, токенизация и т.д.).

In [ ]:
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
import re
from nltk.stem.snowball import SnowballStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##### STEMMING

In [ ]:
stemmer = SnowballStemmer('russian')
nltk.download('stopwords')
stopwords = set(stopwords.words('russian'))

def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    text = word_tokenize(text)
    filtered_tokens = [token for token in text if not token in stopwords]
    text = [stemmer.stem(word) for word in filtered_tokens]

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data_bank_tokenized = data_bank.copy()
data_bank_tokenized['review'] = data_bank_tokenized['review'].apply(preprocess)

data_bank_tokenized['review']

2487     [финанс, плохоб, работ, муж, постоя, доход, бо...
2488     [вчер, закрыва, задолжен, карт, тинькофф, откр...
2489     [клиент, банк, год, им, дебетов, кредитн, карт...
2490     [хорош, надежн, банк, клиент, котор, явля, пер...
2491     [19112016, украд, деньг, карт, пиш, коротк, ср...
                               ...                        
19920    [довольн, клиент, банкаокол, 5ти, лет, явля, к...
19921    [кредитн, карт, tinkoff, bank, платинумпольз, ...
19922    [итак, договор, расторг, 14, ма, 2015, лиш, 2,...
19923    [хамск, отношен, сотрудник, доведен, пожил, че...
19924    [здравствуйтеуважа, сотрудник, банкакт, сможет...
Name: review, Length: 1476, dtype: object

##### LEMMA

Для лемматизации будет использоваться морфологический анализатор для русского языка pymorphy2

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def lemmatize_tokenize_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalnum() and token.lower() not in stop_words]
    lemmatized_tokens = [morph.parse(token)[0].normal_form for token in tokens]

    return lemmatized_tokens

In [ ]:
data_bank_lemma = data_bank.copy()
data_bank_lemma['review'] = data_bank_lemma['review'].apply(
    lambda x: lemmatize_tokenize_text(x))

data_bank_lemma['review']

2487     [финансы, плохобыть, работа, муж, постоянный, ...
2488     [вчера, закрывать, задолженность, карта, тиньк...
2489     [клиент, банк, год, иметь, дебетовый, кредитны...
2490     [хороший, надёжный, банк, клиент, который, явл...
2491     [19112016, украсть, деньга, карта, писать, кор...
                               ...                        
19920    [довольный, клиент, банкаоколо, 5ти, год, явля...
19921    [кредитный, карта, tinkoff, bank, platinumполь...
19922    [итак, договор, расторгнуть, 14, май, 2015, ли...
19923    [хамский, отношение, сотрудник, доведение, пож...
19924    [здравствуйтеуважать, сотрудник, банкакто, смо...
Name: review, Length: 1476, dtype: object

Стемминг в данном случае не показывает хороших результатов по приведению слов к одной форме из-за морфологической структуры русского языка. Поэтому далее в лабораторной будут использоваться результаты, полученные с помощью лемматизации.

#### Найдите Топ-10 наиболее часто используемых слов в отзывах.

In [ ]:
from nltk import FreqDist

In [ ]:
all_words = [word for sublist in data_bank_lemma['review'] for word in sublist]
freq_dist = FreqDist(all_words)
top_words = freq_dist.most_common(10)

print("Топ-10 наиболее часто используемых слов в столбце 'review' в датафрейме:")
for word, count in top_words: print(f"{word}: {count}")

Топ-10 наиболее часто используемых слов в столбце 'review' в датафрейме:
банк: 4630
карта: 2153
тинькофф: 1342
это: 1270
счёт: 1049
мой: 902
день: 890
который: 880
очень: 814
клиент: 791


#### Разделите отзывы на положительные и отрицательные с использованием значения в поле «score» (например, 1,2- негативные, 4,5-позитивные).

In [ ]:
data_bank_neg_pos = data_bank_lemma.copy()

In [ ]:
positive_reviews = data_bank_neg_pos[(data_bank_neg_pos['score'] == '4') | (data_bank_neg_pos['score'] == '5')]
negative_reviews = data_bank_neg_pos[(data_bank_neg_pos['score'] == '1') | (data_bank_neg_pos['score'] == '2')]

print(f"Количество положительных отзывов: {positive_reviews.shape[0]}")
print(f"Количество отрицательных отзывов: {negative_reviews.shape[0]}")

Количество положительных отзывов: 765
Количество отрицательных отзывов: 331


#### Найдите Топ-10 наиболее часто используемых прилагательных, используемых со словом банк для положительных/отрицательных отзывов.


In [ ]:
from collections import Counter

In [ ]:
def extract_adjectives_from_reviews(df):
    adjectives = []
    for review_list in df:
      analyzed = morph.parse(review_list)[0].tag
      if 'ADJF' in analyzed:
        adjectives.append(review_list)

    return adjectives

In [ ]:
positive_adjectives = []
negative_adjectives = []
sub_pos = []
sub_neg = []

for review in positive_reviews['review']:
    if 'банк' in review:
      positive_adjectives.append(extract_adjectives_from_reviews(review))

for ls in positive_adjectives:
  for sub in ls:
    sub_pos.append(sub)

top_positive_adjectives = Counter(sub_pos).most_common(10)

for review in negative_reviews['review']:
    if 'банк' in review:
      negative_adjectives.append(extract_adjectives_from_reviews(review))

for ls in negative_adjectives:
  for sub in ls:
    sub_neg.append(sub)

top_negative_adjectives = Counter(sub_neg).most_common(10)

In [ ]:
print("Топ-10 позитивных прилагательных:")
for word, count in top_positive_adjectives: print(f"{word}: {count}")

print("\n" + "Топ-10 негативных прилагательных:")
for word, count in top_negative_adjectives: print(f"{word}: {count}")

Топ-10 позитивных прилагательных:
который: 335
мой: 292
свой: 259
хороший: 209
весь: 190
кредитный: 178
такой: 172
другой: 172
любой: 162
удобный: 150

Топ-10 негативных прилагательных:
мой: 211
который: 185
кредитный: 170
такой: 148
свой: 144
ваш: 112
какой: 89
один: 87
другой: 82
должный: 65
